# ECON5280 Lecture 6 Least Squares

<font size="5">Junlong Feng</font>

## Outline

* Motivation: We have a linear model. Now what?
* OLS as an MM Estimator: A unified viewpoint to revisit your undergrad econometrics.
* Statistical Properties: Why has OLS been popular for 200 years?
* Inference: Tests and confidence intervals.
* Key Assumptions: What do they mean in theory and in practice?
* When and how to use OLS in practice.

## 1. OLS as an MM Estimator

Throughout this lecture, we focus on the following linear model for an i.i.d. sample $\{Y_{i},X_{i}:i=1,...,n\}$:
$$
Y_{i}=X_{i}'\beta+\varepsilon_{i},\ \ \ \ \mathbb{E}(X_{i}\varepsilon_{i})=0.
$$
Or, stacking all $n$ observations into a vector:
$$
Y=X\beta+\varepsilon,\ \ \ \ \mathbb{E}(X_{i}\varepsilon_{i})=0,\forall i.
$$

* $X_{i}$ is a $k\times 1$ vector which usually contains $1$. 
  * e.g. $X_{i}=(1,D_{i})'$ and $\beta=(\beta_{0},\beta_{1})'$. Then $Y_{i}=\beta_{0}+\beta_{1}D_{i}+\varepsilon_{i}$.
  * Containing constant 1 makes $\mathbb{E}(\varepsilon_{i})=0$ under $\mathbb{E}(X_{i}\varepsilon_{i})=0$.
* $X$ is an $n\times k$ matrix whose $i$-th row is $X_{i}'$. $Y$ and $\varepsilon$ are $n\times 1$ vectors.
* Lecture 5 tells us when $\beta$ is ATE or CATE when $\mathbb{E}(X_{i}\varepsilon_{i})=0$ holds (and possibly under some other assumptions).
* Identification of $\beta$ holds when $\mathbb{E}(X_{i}X_{i}')\neq 0$.
* In this section, we forget about the meaning of $\beta$ and only study how to estimate it when it is identified.

**An MM Estimator**. We have shown in Lecture 5 that $\beta=[\mathbb{E}(X_{i}X_{i}')]^{-1}\mathbb{E}(X_{i}Y_{i})$ under $\mathbb{E}(X_{i}\varepsilon_{i})=0$ and under the identification condition $\mathbb{E}(X_{i}X_{i}')$ being full rank. However, these expectations are population quantities which we don't know how to calculate without knowing $(X_{i},Y_{i})$'s joint distribution.

* $\mathbb{E}(X_{i}X_{i}')=\int xx'f_{X}(x)dx$ and $\mathbb{E}(X_{i}Y_{i})=\int xyf_{XY}(x,y)dxdy$.

Now we use the principle of MM estimator to estimate $\beta$. The idea is simple: We can't compute the expectations, so we replace them with sample averages:
$$
\begin{align*}
\hat{\beta}\equiv &\left[\frac{1}{n}\sum_{i=1}^{n}X_{i}X_{i}'\right]^{-1}\left[\frac{1}{n}\sum_{i=1}^{n}X_{i}Y_{i}\right]\\
=& (X'X)^{-1}(X'Y).
\end{align*}
$$

* $\hat{\beta}$ is indeed an **estimator** because it solely depends on sample/data: Once you have data, no more unknowns in the formula and you can calculate $\hat{\beta}$ directly.
* This derivation of $\hat{\beta}$ is from the angle of an MM estimator; that is, constructing moment conditions first ($\mathbb{E}[X_{i}(Y_{i}-X_{i}'\beta)]=0$, solve for $\beta$, and substitute all expectations with sample averages.
* This estimator has another name: Ordinary Least Squares (OLS), motivated from a different prospective (independently developed by Legendre and Gauss 200 years ago), widely adopted in traditional econometric texts. We did not derive OLS in that way since it's not general enough to nest other estimators we'll learn in this semester. 
* Note the model is linear in $\beta$ but not necessarily linear in components in $X$. For instance, the estimator can handle anything like $Y_{i}=\beta_{0}+\beta_{1}D_{i}+\beta_{2}D_{i}^{2}+\beta^{3}\log D_{i}+\varepsilon_{i}$.

In [ ]:
### Verify that the function lm in R gives the same estimates as the formula
n=100; e=rnorm(n,0,1); x=rnorm(n,0,1); y=0.5+2*x+e; cons=rep(1,100)
X=cbind(cons,x); y=as.matrix(y,n,1)
## Method 1: Use formula
bhat=solve(t(X)%*%X)%*%(t(X)%*%y)
## Method 2: Use R routine
model=lm(y~x)
summary(model)

### 1.1 Fitted Value and Residuals

The linear model essentially estimates a *line* (or, hyperplane more accurately) to fit the data points of $Y$. We have the following jargons:

* **Fitted or predicted value**: $\hat{Y}\equiv X\hat{\beta}=X(X'X)^{-1}X'Y$.
* **Residual**: $\hat{\varepsilon}\equiv Y-X\hat{\beta}=(I-X(X'X)^{-1}X')Y$.
* **Projection matrix**: $P=X(X'X)^{-1}X'$.
* **Elimination matrix**: $M=I-P$.

**Some insights**. The projection matrix **linearly** projects $Y$ onto the space of $X$. The projection, i.e., $\hat{Y}$, automatically lies orthogonal to the residual $\hat{\varepsilon}$:
$$
\begin{align*}
P=P'\text{ (symmetric) },\ \ \ \ &P^{2}=X(X'X)^{-1}X'X(X'X)^{-1}X'=X(X'X)^{-1}X'=P\text{ (idempotent)}\\
\implies(PY)'(MY)&=Y'P'MY=Y'(P-P^{2})Y=0\implies \mathbb{E}[(PY)'(MY)]=0.
\end{align*}
$$
Moreover, the residuals **do not contain any (linear) information in $X$**: $P\hat{\varepsilon}=PMY=0$.

Note these results hold algebraically, **not depending on the assumption $\mathbb{E}(X_{i}\varepsilon_{i})=0$**.

In [ ]:
### Verify orthogonality between projection and the residual
n=100; e=rnorm(n,0,1); x=rnorm(n,0,1); y=0.5+2*x+e; cons=rep(1,n)
X=cbind(cons,x); y=as.matrix(y,n,1)
P=X%*%solve(t(X)%*%X)%*%t(X);   I=diag(1,n,n);   M=I-P
sum((P%*%M)^2)

**Geometric interpretation**.

**Remark **. We derived the OLS estimator from the exogeneity assumption $\mathbb{E}(X_{i}\varepsilon_{i})=0$, but the formula $\hat{\beta}=(X'X)^{-1}(X'Y)$ itself exists by its own right. We have shown that the estimator mechanically leads to orthogonal fitted value and residuals, or in other words, the residuals are NOT correlated with $X$. So by reverse engineering, we should expect the estimator works well when the true unobservable $\varepsilon$ is uncorrelated with $X$, and works poorly when $\varepsilon$ is correlated with $X$. Now we formally justify this point.

## 2. Statistical Properties of OLS

To derive the properties of OLS, it's useful to observe that
$$
\hat{\beta}=(X'X)^{-1}(X'Y)=(X'X)^{-1}(X'X\beta+X'\varepsilon)=\beta+(X'X)^{-1}X'\varepsilon.
$$

### 2.1 Unbiasedness

Unbiasedness of OLS holds under a stronger assumption $\mathbb{E}(\varepsilon_{i}|X_{i})=0$.

* Recall this holds under complete random assignment when $\varepsilon_{i}$ is a function of the underlying unobservables $U_{i}$.

Proof. We are done once we show that $\mathbb{E}(\hat{\beta}|X)=0$ because then $\mathbb{E}(\hat{\beta})=0$ by LIE. 
$$
\begin{align*}
\mathbb{E}(\hat{\beta}|X)=\mathbb{E}(\beta+(X'X)^{-1}X'\varepsilon|X)=\beta+(X'X)^{-1}X'\mathbb{E}(\varepsilon|X)=0.
\end{align*}
$$

In [ ]:
### Verify Unbiasedness
n=100; nrep=500 # Create nrep samples and in each sample estimate beta=(0.5,2).
bhat=matrix(0,2,nrep)
for (sim in 1:nrep){
e=rnorm(n,0,1); x=rnorm(n,0,1); y=0.5+2*x+e; cons=rep(1,n)
X=cbind(cons,x); y=as.matrix(y,n,1)
bhat[,sim]=solve(t(X)%*%X)%*%(t(X)%*%y)
}
rowMeans(bhat) # Take average which mimics E(bhat).

### 2.2 Consistency

By $X'X=\sum_{i}X_{i}X_{i}'$ and $X'\varepsilon=\sum_{i}X_{i}\varepsilon_{i}$, we have 
$$
\hat{\beta}=\beta+\left(\sum_{i}X_{i}X_{i}'\right)^{-1}\left(\sum_{i}X_{i}\varepsilon_{i}\right)=\beta+\left(\frac{1}{n}\sum_{i}X_{i}X_{i}'\right)^{-1}\left(\frac{1}{n}\sum_{i}X_{i}\varepsilon_{i}\right).
$$
We establish consistency by the following argument:

* By WLLN, $\frac{1}{n}\sum_{i}X_{i}X_{i}'\to_{p}\mathbb{E}(X_{i}X_{i}')$ and $\frac{1}{n}\sum_{i}X_{i}\varepsilon_{i}\to_{p}\mathbb{E}(X_{i}\varepsilon_{i})$.
* By invertibility of $\mathbb{E}(X_{i}X_{i}')$ and by continuous mapping theorem, $(\frac{1}{n}\sum_{i}X_{i}X_{i}')^{-1}\to_{p}[\mathbb{E}(X_{i}X_{i}')]^{-1}$.
* By continuous mapping theorem again, $(\frac{1}{n}\sum_{i}X_{i}X_{i}')^{-1}(\frac{1}{n}\sum_{i}X_{i}\varepsilon_{i})\to_{p}[\mathbb{E}(X_{i}X_{i}')]^{-1}\mathbb{E}(X_{i}\varepsilon_{i})$, where the right hand side is 0 by $\mathbb{E}(X_{i}\varepsilon_{i})=0$.
* Therefore, $\hat{\beta}\to_{p}\beta$.

In [ ]:
### (Not rigorously) Verify Consistency
set.seed(5280)
n0=c(50,100,1000); # Create three samples with n=50, 500, 1000.
diff=matrix(0,2,3)
for (sim in 1:3){
  n=n0[sim]
e=rnorm(n,0,1); x=rnorm(n,0,1); y=0.5+2*x+e; cons=rep(1,n)
X=cbind(cons,x); y=as.matrix(y,n,1)
diff[,sim]=solve(t(X)%*%X)%*%(t(X)%*%y)-matrix(c(0.5,2),2,1) # difference between bhat and b
}
colSums(abs(diff))

### 2.3 Asymptotic Normality

Again, by $\hat{\beta}-\beta=\left(\frac{1}{n}\sum_{i}X_{i}X_{i}'\right)^{-1}\left(\frac{1}{n}\sum_{i}X_{i}\varepsilon_{i}\right)$, we have
$$
\begin{align*}
\sqrt{n}(\hat{\beta}-\beta)=&\left(\frac{1}{n}\sum_{i}X_{i}X_{i}'\right)^{-1}\left(\frac{1}{\sqrt{n}}\sum_{i}X_{i}\varepsilon_{i}\right)\\
\text{(by i.i.d. and by $\mathbb{E}(X_{i}\varepsilon_{i})=0$)  }=&\left(\frac{1}{n}\sum_{i}X_{i}X_{i}'\right)^{-1}\left(\sqrt{n}\cdot\left[\frac{1}{n}\sum_{i}X_{i}\varepsilon_{i}-\mathbb{E}(X_{i}\varepsilon_{i})\right]\right).\\
\end{align*}
$$
We establish asymptotic normality by the following argument:

* By WLLN, CMT, and invertibility of $\mathbb{E}(X_{i}X_{i}')$, $\left(\frac{1}{n}\sum_{i}X_{i}X_{i}'\right)^{-1}\to_{p}[\mathbb{E}(X_{i}X_{i}')]^{-1}$.
* By CLT, $\sqrt{n}\cdot\left[\frac{1}{n}\sum_{i}X_{i}\varepsilon_{i}-\mathbb{E}(X_{i}\varepsilon_{i})\right]\to_{d}N(0,V(X_{i}\varepsilon_{i})).$
* By $\mathbb{E}(X_{i}\varepsilon_{i})=0$, $V(X_{i}\varepsilon_{i})=\mathbb{E}(\varepsilon_{i}^{2}X_{i}X_{i}')$.
* By CMT, $\sqrt{n}(\hat{\beta}-\beta)\to_{d}N(0,[\mathbb{E}(X_{i}X_{i}')]^{-1}\mathbb{E}(\varepsilon_{i}^{2}X_{i}X_{i}')\mathbb{E}(X_{i}X_{i}')]^{-1})$.
* For simplicity we denote $\Sigma\equiv [\mathbb{E}(X_{i}X_{i}')]^{-1}\mathbb{E}(\varepsilon_{i}^{2}X_{i}X_{i}')\mathbb{E}(X_{i}X_{i}')]^{-1}$.

Asymptotic normality says, although we know nothing about the distribution of data $\{(Y_{i},X_{i}):i=1,...,n\}$, and thus know nothing about the exact distribution of $\hat{\beta}$, we can approximate its distribution by $N(\beta,\Sigma/n)$ when $n$ is large.

* The variance $\Sigma$ is called the **asymptotic variance** of $\hat{\beta}$. 
* We usually call a formula like $\Sigma$ **sandwich formula**: $[\mathbb{E}(X_{i}X_{i}')]^{-1}$ is bread and $\mathbb{E}(\varepsilon_{i}^{2}X_{i}X_{i}')$ meat.
* This formula can be simplified under **an additional assumption:  homoscedasticity**.

**Definition**. Homoscedasticity: $\mathbb{E}(\varepsilon_{i}^{2}|X_{i})=\mathbb{E}(\varepsilon_{i}^{2})$. Heteroscedasticity: $\mathbb{E}(\varepsilon_{i}^{2}|X_{i})\neq\mathbb{E}(\varepsilon_{i}^{2})$.

* Homo means the squared error term $\varepsilon_{i}^{2}$ is conditional mean idenpendent of $X_{i}$; it is **not** implied by $\mathbb{E}(\varepsilon_{i}|X_{i})=0$ but **is implied** by $\varepsilon_{i}\perp X_{i}$.  
* Homo says the **conditional** variance of $\varepsilon_{i}$ does not depend on $X_{i}$. It **does not say** anything about the **unconditional** variance of $\varepsilon_{i}$. The latter is always a constant by i.i.d.

**Claim**. Let $\sigma^{2}\equiv \mathbb{E}(\varepsilon_{i}^{2})$. Assuming homoscedasticity holds, $\Sigma$ can be simplified as $\Sigma=\sigma^{2}[\mathbb{E}(X_{i}X_{i}')]^{-1}$.

#### 2.3.1 Estimating $\Sigma$

From Lecture 4, we want to know the asymptotic distribution because we want to conduct inference: Recall when we used $\bar{X}$ to estimate the population mean $\mu$, we constructed t-test statistic using $\sqrt{n}(\bar{X}-\mu^{0})/\hat{\sigma}$ because we know its distribution is approximately $N(0,1)$. Similarly, we constructed $(1-\alpha)$ confidence interval by $[\hat{\mu}-z_{1-\alpha}se,\hat{\mu}+z_{1-\alpha}se]$ where $se=\hat{\sigma}/\sqrt{n}$.

Here again, if we hope to conduct inference about $\beta$, we need distributional information of $\hat{\beta}$. Now that we already derived its asymptotic distribution, the only missing part is a consistent estimator of the asymptotic variance $\Sigma$. 

There are two difficulties in it: i) $\mathbb{E}$ is not computable because again, we don't know the distribution of data, ii) $\varepsilon_{i}$ is unknown. For i), we are now very familiar with it: We can simply replace all $\mathbb{E}$ by sample averages and WLLN guarantees consistency. For ii), we can replace $\varepsilon_{i}$ with the residuals $\hat{\varepsilon}_{i}$.

* **Heteroscedastic robust variance estimator**: $\hat{\Sigma}\equiv (\sum_{i}X_{i}X_{i}'/n)^{-1}(\sum_{i}X_{i}X_{i}'\hat{\varepsilon}_{i}^{{2}}/n)(\sum_{i}X_{i}X_{i}'/n)^{-1}$.
  * This one is **ALWAYS** consistent of $\Sigma$ under either homo or hetero.
  * That's why it's called *robust*.
* **Homoscedastic only variance estimator**: $\hat{\Sigma}^{homo}\equiv (\sum_{i}\varepsilon_{i}^{2}/n)(\sum_{i}X_{i}X_{i}'/n)^{-1}$.
  * This one is **ONLY** consistent of $\Sigma$ under homo.

In the past, people do something called *homoscedasticity test* to see whether homo holds or not in the first, and if homo holds, use homo variance estimator. There are several drawbacks in this approach.

* The power of such tests are usually low, meaning that if you cannot reject, it's still highly possible that the null (which is unfortunately homo) is false. When it happens, you end up with a wrong variance estimator.
* Such pre-testing leads to a problem called **multitesting**, leading to poor size control when you begin to run the regression and test hypotheses about $\beta$.

So today, people seldom test homo and just use $\hat{\Sigma}$ in all scenarios because it's always correct.

**Standard error of $\hat{\beta}$**.

* The diagonal elements in $\hat{\Sigma}$ are estimators of the asymptotic variances of each $\hat{\beta}_{j}$ in the $k\times 1$ vector $\hat{\beta}$. Denote them by $\hat{\Sigma}_{jj}$s.
* We call $\sqrt{\hat{\Sigma}_{jj}}/\sqrt{n}$ the **standard error** of $\hat{\beta}_{j}$, denoted by $se_{j}$.
* By consistency of $\hat{\Sigma}$ and the asymptotic distribution of $\hat{\beta}$, we have $(\hat{\beta}_{j}-\beta_{j})/se_{{j}}\to_{d}N(0,1)$. This is the foundation of inference.

In [ ]:
### Illustration of asymptotic variance and homoscedasticity
n=500; e=rnorm(n,0,1); x=rnorm(n,0,1); y=0.5+2*x+e; cons=rep(1,n) # This DGP is homo
X=cbind(cons,x); y=as.matrix(y,n,1)

## Method 1. Calculate the hetero-robut se by formula
bhat=solve(t(X)%*%X)%*%(t(X)%*%y)
res=y-X%*%bhat
res_sq_matrix=diag(c(res^2),n,n)
Sigma_robust=solve(t(X)%*%X/n)%*%(t(X)%*%res_sq_matrix%*%X/n)%*%solve(t(X)%*%X/n)
se=sqrt(diag(Sigma_robust))/sqrt(n)
se
## Method 2. Calculate the homo only se.
model=lm(y~x)
summary(model) # "summary" always assumes homo and calculates the homo-only se

## 3 Inference

### 3.1 Inference about a Single Parameter

By $(\hat{\beta}_{j}-\beta_{j})/se_{{j}}\to_{d}N(0,1)$, we have the following results by the same argument as in Lecture 4: 

* Testing. T-test: For $\mathbb{H}_{0}:\beta_{j}=\beta_{j}^{0}$ vs $\mathbb{H}_{1}:\beta_{j}\neq \beta_{j}^{0}$, we construct $T_{n}\equiv (\hat{\beta}_{j}-\beta_{j}^{0})/se_{j}$. To achieve $\alpha$ level, reject when $|T_{n}|>z_{1-\alpha/2}$ where $z_{1-\alpha/2}$ is the $(1-\alpha/2)$-th quantile of $N(0,1)$.
  * $p$-value. Suppose the realization of $T_{n}$ in a particular data set is $t$. Then $p=\mathbb{P}(|Z|> |t|)$ where $Z\sim N(0,1)$. Reject if $p<\alpha$.
* Confidence interval. An $(1-\alpha)$ confidence interval is $[\hat{\beta}_{j}-se_{j}\times z_{1-\alpha/2},\hat{\beta}_{j}+se_{j}\times z_{1-\alpha/2}]$.
* $se_{j}$ are calculated from $\hat{\Sigma}$ or $\hat{\Sigma}^{homo}$, depending on the assumption on homoscedasticity.

**Important**. Recall that by construction $\sqrt{n}se_{j}\equiv\hat{\Sigma}_{jj}\to_{p}\Sigma_{jj}$, a positive constant. Hence, $se_{j}\to 0$ as $n\to\infty$! That means, you can literally reject almost any null when $n$ is sufficiently large! Having statistical significance is not a big deal at all.

* You can verify this by modifying $n$ in the previous R example.
* This result looks striking, but is natural if we really understand what **significance** means. For instance, suppose our estimate is $0.01$. We want to test whether the true parameter is 0. Obviously 0.01 is not equal to 0 mathematically. But you also have to take sampling error into consideration. When your sample is small, you don't have enough info, so you're not very confident and you're afraid you get 0.01 by making a mistake. So you say well maybe I cannot reject 0 because although $0.01\neq 0$, my info is not enough so I may make a mistake so 0 is possibly to be the true parameter. However, when you have, let's say 100 million data points and you still get an estimate equal to 0.01, then you may confidently say that the truth is not 0 because I got this number based on such rich info.

### 3.2 Inference about Multiple Parameters and Function(s) of Parameters

Sometimes one can be interested in multiple parameters or functions of parameters $r(\beta):\mathbb{R}^{k}\to\mathbb{R}^{q}$, where $k$ is the number of coefficients in the regression and $q$ is the number of restrictions you want to test.

- $Income=\beta_{0}+\beta_{1}College+\beta_{2}Graduate+\varepsilon$. $\beta_{0}$: mean income of someone who doesn't receive any post-secondary eduaction. $\beta_{1}$: income gain by attending college. $\beta_{2}$: income gain by attending graduate school.
- One may ask: Does graduate school have equal income effect as college? (If the answer is yes and if one attends graduate school only to increase her/his income, then it will be a waste of time and money...)
  - $\mathbb{H}_{0}:\beta_{1}-\beta_{2}=0\ vs\ \mathbb{H}_{1}:\beta_{1}-\beta_{2}\neq 0$. 
  - This is an example of testing a function of parameters: $r(\beta)=\beta_{1}-\beta_{2}$. 
  - $k=3,q=1$.
- One may ask: Does receiving post-secondary education has any effect on income at all?
  - $\mathbb{H}_{0}:\beta_{1}=\beta_{2}=0,\ vs\ \mathbb{H}_{1}:\text{At least one of $\beta_{1}$ and $\beta_{2}$ is not 0}$. 
  - This is an example of testing multiple hypotheses: the null consists of two hypotheses $\beta_{1}=0$ and $\beta_{2}=0$. But it can also be viewed as testing two functions of parameters $r(\beta)=(\beta_{1},\beta_{2})'$.
  - We can also equivalently view the null as $\beta_{1}-\beta_{2}=0$ and $\beta_{1}=0$. $r(\beta)=(\beta_{1}-\beta_{2},\beta_{1})$.
  - Either way, we have $k=3,q=2$.
- Testing one parameter, say $\beta_{j}=0$, is a special case where $k=3,q=1$, and $r(\beta)=\beta_{j}$.

Intuition. If the null is true, i.e., $r(\beta)=0$, then $r(\hat{\beta})$ must be close to 0. If $r$ is one dimensional, i.e., if $q=1$, then we can simply compare its absolute value with 0. Of course, to know how close is close, we should have a criterion, which is called the *critical value*, obtained from the asymptotic distribution of a re-scaled $r(\hat{\beta})$. This is the logic of t-test.

If, on the other hand, $q>1$, for instance $r(\beta)=(\beta_{1},\beta_{2})'$, how do we determine whether it is small or not?

- Euclidean distance: $\sqrt{\beta_{1}^{2}+\beta_{2}^{2}}$. $(\beta_{1},\beta_{2})'=0$ if and only if $\beta_{1}^{2}+\beta_{2}^{2}=0$.
- Inner product:  the squared Euclidean distance is indeed the inner product $r(\beta)'r(\beta)=\beta_{1}^{2}+\beta_{2}^{2}$. So the null is true if and only if $r(\beta)'r(\beta)=0$, or equivalently, $r(\beta)'Ir(\beta)=0$ where $I$ is the $q\times q$ identity matrix.
- Weighted inner product: For a positive definite (p.d.) matrix $W$, $r(\beta)=0$ if and only $r(\beta)'Wr(\beta)=0$.
- So, we can find some p.d. matrix so that $r(\hat{\beta})'Wr(\hat{\beta})$ has a known asymptotic distribution. Then we find a critical value from the distribution to control the probability of type one error, and compare $r(\hat{\beta})'Wr(\hat{\beta})$ with the critical value. If it's greater than the critical value, then we reject the null.

To find such a $W$, we need to know a few things:

- $\chi_{q}^{2}$  distribution: If $Z\equiv (Z_{1},\ldots,Z_{q})$ and $Z\sim N(0,I)$ where $I$ is the $q\times q$ identity matrix, then the distribution of $Z'Z$ is called $\chi_{q}^{2}$.

- Delta method: If $\sqrt{n}(\hat{\beta}-\beta)\to_{d}N(0,\Sigma)$, then $\sqrt{n}(r(\hat{\beta})-r(\beta))\to_{d}N(0,\partial_{\beta'}r(\beta)\Sigma \partial_{\beta}r(\beta))$ where $\partial_{\beta'}r(\beta)$ is the $q\times k$ Jacobian matrix of $r(\beta)$. Two examples of Jacobian:

  -  $r(\beta)=(\beta_{1},\beta_{2})'$. Then $\partial_{\beta'}r(\beta)=\begin{pmatrix}0&1&0\\0&0&1\end{pmatrix}$.
  -  $r(\beta)=(\beta_{1}-\beta_{2},\beta_{1})'$. Then $\partial_{\beta'}r(\beta)=\begin{pmatrix}0&1&-1\\0&1&0\end{pmatrix}$.
  -  $r(\beta)=(\beta_{1}^{2}+\beta_{2},\beta_{0}+\log(\beta_{2}))$. Then $\partial_{\beta'}r(\beta)=\begin{pmatrix}0&2\beta_{1}&1\\1&0&1/\beta_{2}\end{pmatrix}$.
  -  Rank of the Jacobian should be $q$ because otherwise at least one of the null is redundant.

- Let $\Omega=\partial_{\beta'}r(\beta)\Sigma \partial_{\beta}r(\beta)$. By full-rankness of the Jacobian and $\Sigma$, and by this quadratic form, $\Omega$ is p.d. Therefore, under the null $r(\beta)=0$, $\sqrt{n}\Omega^{-1/2}r(\hat{\beta})\to_{d}N(0,I)$ and thus, 
  $$
  \left[\sqrt{n}\Omega^{-1/2}r(\hat{\beta})\right]'\left[\sqrt{n}\Omega^{-1/2}r(\hat{\beta})\right]=n \cdot r(\hat{\beta})'\Omega^{-1}r(\hat{\beta})\to_{d}\chi_{q}^{2}.
  $$

- $\Omega$ involves a lot of unknowns: $\Sigma$ is unknown and the Jacobian can also contain the unknown parameters (see the third example above). In practice, we need to estimate them: $\hat{\Omega}=\partial_{\beta'}r(\hat{\beta})\hat{\Sigma}\partial_{\beta}r(\hat{\beta})$. We already know what is $\hat{\Sigma}$. 

The statistic $n \cdot r(\hat{\beta})'\hat{\Omega}^{-1}r(\hat{\beta})$ is called **Wald statistic**, denoted by $W_{n}$. The procedure of comparing it with the $(1-\alpha)$-th quantile of $\chi_{q}^{2}$ is called **Wald test**.

- Obtain $\hat{\beta}$ and $\hat{\Sigma}$. Usually use the heteroscedastic-robust version for the latter.
- Based on your $r$, compute $r(\hat{\beta})$ and $\partial_{\beta'}(\hat{\beta})$.
- Plug every component into the formula of $W_{n}$. Compare it with critical value, i.e., the $(1-\alpha)$-th quantile of $\chi_{q}^{2}$. 
- If $W_{n}$ is greater than the critical value, reject the null $r(\beta)=0$. If $W_{n}$ is smaller than the critical value, cannot reject.

Final remarks. Wald test is so general that you can use it to test almost all kinds of equality nulls, no matter whether $r$ is one-dimensional or multidimensional, and no matter whether the functions in $r$ are linear or nonlinear. It also nests t-test: Since t-test the case where $q=1$, the square of one standard normal is a $\chi_{1}^{2}$ distribution, you can verify that the square of the t statistic is equal to the Wald statistic in that case, and the critical value of Wald is exactly equal to the square of critical value of the t test.

In [ ]:
print(qchisq(0.95,1))
print(qnorm(0.975,0,1)^2)

An R example:

In [ ]:
library(sandwich) # library to calculate heteroscedastic robust standard error
library(nlWaldTest) # library to do Wald
# DGP
n=500; e=rnorm(n,0,1); x1=rnorm(n,0,1); x2=rnorm(n,0,1); y=0.5+x1+x2+e; cons=rep(1,n) 
model=lm(y~x1+x2)
## Get Hetero-robust variance matrix. Any of HC0-HC3 is okay. HC0 and 1 are commonly used.
cov=vcovHC(model,type="HC0")
## First test the coefficient on x1 is 0 using t-test
ttest1=model$coefficient[2]/sqrt(cov[2,2]) # denominater is the corresponding se.
print(ttest1^2)
## Or you can use command coeftest:
coeftest(model,vcov.=vcovHC(model,type="HC0"))
## Test the same hypothesis by Wald
Wald1=nlWaldtest(model,"b[2]",Vcov=cov)
print(Wald1)
## Now test the joint hypotheses: coef. on x1 and x2 are equal & coef on x1 is 0
Wald2=nlWaldtest(model,c("b[2]-b[3]","b[2]"),Vcov=cov)
print(Wald2)
## Test another joint hypotheses: coef. on x1 and x2 are equal & coef on x1 is 1
Wald3=nlWaldtest(model,c("b[2]-b[3]","b[2]^2=1"),Vcov=cov)
print(Wald3)

## 4 Revisit the Key Assumption

### 4.1 $\mathbb{E}(X_{i}\varepsilon_{i})=0$

In Lecture 5, we discussed assumptions under which ATE or CATE can be identified by parameters in a linear model. In this lecture we studied assumptions under which parameters in a linear model can be consistently estimated. Are they the same? Short answer: No.

Suppose we have a dummy treatment $D_{i}$. We know from Lecture 5 that without loss of generality, we always write
$$
Y_{i}=\beta_{0}+\beta_{1}\times D_{i}+\varepsilon_{i},\ \ \beta_{1}=ATE.
$$
We show that identification of $\beta_{1}$ (Lecture 5) and consistent estimation of $\beta_{1}$ (this lecture) both require $\mathbb{E}(\varepsilon_{i}D_{i})=0$. However, if $D$ is not determined by an experiment, how is this possible?

- One may worry $\varepsilon$ may contain variables that are correlated with $D$. For instance, $D$ is whether you attend to UST or not, then things like college, major, family income etc. may be in $\varepsilon$.

- Mathematically, that means $\varepsilon=h(W,\eta)$ where $W$ is a vector that contains all possible missing variables. Then
  $$
  Y_{i}=\beta_{0}+\beta_{1}\times D_{i}+h(W_{i},\eta_{i}),\ \ \beta_{1}=ATE.
  $$

- **If you're willing to assume linearity** in $h$, then 
  $$
  Y_{i}=\beta_{0}+\beta_{1}\times D_{i}+W_{i}'\gamma+\eta_{i},\ \ \beta_{1}=ATE.
  $$

- Now maybe it's more reasonable to believe $\mathbb{E}(\eta_{i}D_{i})=0$ because $\eta$ no longer contains the confounding part $W$.

- **However**, consistent estimation of $\beta_{1}$ of OLS requires $\mathbb{E}(\eta_{i}W_{i})=0$ as well. 

  - Two exceptions. i) $D_{i}$ and $W_{i}$ are independent (but in that case, leaving $W$ in the error does not cause inconsistency in the first place). ii）$\mathbb{E}(\eta_{i}|W_{i})=W_{i}'\delta$ for some $\delta$, which you have to assume without any guarantee.

- **Dilemma**: 

  - If we do not control for $W$ (i.e., do not include $W$ in the regression), we may have inconsistent estimator because $W$ is then implicitly in the error $\varepsilon$, making $\mathbb{E}(W\varepsilon)=0$ fail. This is called **omitted variable bias**.
  - If we do control for $W$, we may still obtain inconsistent estimate of $\beta_{1}$ because $W$ may be correlated with $\eta$ and $D$ and $W$ are correlated as well.

**Takeaway**. Adding controls is a very common effort to achieve clean identification and consistent estimation in practice. However, it does not always work for OLS. Introducing controls may introduce new problems into the regression because they themselves may be correlated with the new error, messing up all estimates. This strategy works only under further assumptions as mentioned in the "Two exceptions" earlier.

### 4.2 Invertibility of $\mathbb{E}(X_{i}X_{i}')$

We required $\mathbb{E}(X_{i}X_{i}')$ to be invertible for both identification and consistent estimation.

- Identification: $\beta=[\mathbb{E}(X_{i}X_{i}')]^{-1}\mathbb{E}(X_{i}Y_{i})$.
- Consistent estimation: $\hat{\beta}=(X'X)^{-1}(X'Y)$. We didn't assume $XX'$ to be invertible but by $XX'/n\to_{p}\mathbb{E}(X_{i}X_{i}')$ so invertibility of the latter implies invertibility of $X'X$ when $n$ is sufficiently large.

Violation of this assumption is usually called **perfect multicollinearity**. 

- $X_{i}$ is called perfect multicollinear if there exists a vector $\delta$ such that $X_{i}'\delta=0$ for all $i$. 
- When this happens, $\delta'\mathbb{E}(X_{i}X_{i}')\delta=\mathbb{E}[(X_{i}'\delta)'(X_{i}'\delta)]=0$, implying that $\mathbb{E}(X_{i}X_{i}')$ is not full rank.
- Example: dummy variable trap. Suppose $X=(1,M,F,O)$ where $M,F,O$ are three dummies. $M=1$ if the gender is male. $F=1$ if gender is female. $O=1$ for any other cases. Then you can verify $\delta=(-1,1,1,1)'$ because $M+F+O=1$.
  - Dropping any of the 4 variables solves the problem.

A more common case is the so called **imperfect multicollinearity**, i.e., components in $X$ are highly correlated, making $\mathbb{E}(X_{i}X_{i}')$ close to non-invertible.

- This will NOT invalidate consistency or identification.
- But this will give you a large se, making your estimates less likely to be significant (because se is in the denominator of the t-stat).
- To see this, recall that $\sqrt{n}se\to_{p} [\text{diag}(\Sigma)]^{1/2}$. $\Sigma=[\mathbb{E}(X_{i}X_{i}')]^{-1}\mathbb{E}(\varepsilon_{i}^{2}X_{i}X_{i}')[\mathbb{E}(X_{i}X_{i}')^{-1}]$. When $\mathbb{E}(X_{i}X_{i}')$ is close to 0 (in matrix sense; more rigorously, when its smallest eigenvalue is close to 0), it will in general make the inverse large (in matrix sense), leading to a large $\Sigma$, and thus a large se.

So in practice, when you do find the parameter of interest is not significant, you can:

- Regress your key variable $D$ on to the controls $W$ and see which controls are mostly correlated with your key variable. Call them $W_{1}$ and the rest of controls in $W_{2}$.
- Conduct a Wald test in the main regression ($Y$ on to $(D,W_{1},W_{2})$) for the coefficients on $(D,W_{1})$. If they are jointly significant, it suggests maybe the insignifcant estimate of the coefficient on $D$ is due to large standard error by imperfect multicollinearity between $D$ and $W_{1}$.
- Drop $W_{1}$ and regress $Y$ onto $(D,W_{2})$. See how results change.

  - Larger bias because $W_{1}$ are highly correlated with $D$ and now in error.
  - Smaller variance because imperfect multicollinearity is mitigated.
  - Bias-variance trade off.
  - Can report both results and make a discussion.
- Example. Suppose you have one control $W$ and you run OLS and get $\hat{Y}=1+2D+W$ with $se(\hat{\beta}_{D})=1.5$. 
  - Not significant because $2/1.5<1.64<1.96$. 
  - Now you regress $D$ on $W$ and find the estimate of the coefficient of $W$ is significant.
  - These hint $\mathbb{E}(X_{i}X_{i}')$ is close to 0 ($X_{i}=(1,D_{i},W_{i})'$).
  - Further you do Wald with $\mathbb{H}_{0}:\beta_{D}=\beta_{W}=0$. You find the result is indeed significant.
  - Finally regress $Y$ onto $D$ only and obtain  $\hat{Y}=1.2+1.4D$ with $se(\hat{\beta}_{D})=0.7$.
  - This time it's significant. 1.4 can be more biased than 2 because $W$ is omitted into the error but we know $W$ is correlated with $D$. However, the bias is downward towards 0, and you get a significant result for this smaller estimate, then you can argue that 2, closer to the truth, should also be significantly different from 0.
  - This argument is **NOT** rigorous and can be totally wrong! For instance, how can you be sure that 2 is closer to the truth? What if it's inconsistent as well because $W$ is correlated with the error as we discussed in Section 4.1. However, as long as you're careful in this analysis and provide enough evidence supporting your argument, sometimes readers buy your story. This is the art side of applied econometrics.

## 5 Using OLS in Real World Causal Problems

See Table 3.2.1 in Mostly Harmless Econometrics.

* What's the potential problem for Column (1)?
  * Omitted variables:
* Do you think adding controls solve the problem?
* Would these controls cause new problems?

**In practice**, I have the following recommendations:

- If the variable of interest, or key variable, or treatment, $D$, comes from a random trial/experiment, say a lottery, and is dummy, write down a linear model and run OLS **if you want to learn ATE** only.
  - Usually you run two versions: $Y$ on $D$ alone, and $Y$ on $D$ and controls $W$.
  - Controls are included NOT for consistency; $D$ is already independent of everything else. 
  - The purpose of adding controls is to i) show robustness (i.e., estimates do not change across the two versions so $D$ is indeed independent of everything else) and ii) to reduce standard errors (the error is less noisy so $\Sigma$ becomes smaller (in matrix sense)).
  - Estimate of the coefficient of $D$ is consistent even if $W$ is correlated with the error because $D$ is independent of $W$.
  - CATE usually cannot be consistently estimated in this way because linear model is usually wrong when we think about CATE. See Lecture 5.
- In all other cases, writing down a linear model is already an assumption and including controls can or cannot resolve the issue of omitted variable bias. The best you can do if you do want to use OLS is the following: By carefully comparing differences across specifications, you come up with a logical story to convince your readers. See the earlier real data example.
  - We will learn modern approaches (e.g. machine learning) which resolve these problems in Lecture 8.
- When you have an insignicant estimate, don't panic. 
  - Significance is no longer the first order priorty in today's econ research. Top journals discourage researchers to report stars and $p$-values.
  - Sample size is the most important factor for (in)significance. So if your result is not significant, blame your sample size first.
    - Instead of saying it's not significant, say "my data set is not rich enough to obtain a precise estimate of the effect".
  - When your sample size is reasonable, insignificance may be a result of imperfect multicollinearity. You may consider to do the procedures in Section 4.2.
- To sum up, OLS works under strong assumptions $\mathbb{E}(X_{i}\varepsilon_{i})=0$ (and linearity). This is the last method you want to use unless you really have experimental data, your $D$ is non-continuous, and you only care about ATE.